In [34]:
using LinearAlgebra
A = [1 4 1;
     4 3 2;
     1 2 5]

3×3 Matrix{Int64}:
 1  4  1
 4  3  2
 1  2  5

In [35]:
eigen(A)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -2.1529137088817887
  3.321327095067036
  7.831586613814752
vectors:
3×3 Matrix{Float64}:
  0.77535   -0.428782  -0.463658
 -0.627948  -0.445339  -0.638243
  0.067182   0.786014  -0.614547

In [36]:
function calculate_max_eigen_value(x::Vector, A::Matrix)
    return x'*A*x
end

function power_max_eigen(A::Matrix, acc=1e-10::Float64)::Vector
    n = size(A, 1)
    x = [1:n;];
    max_eigen_value = 1.0;
    
    while true
        prev_max_val = calculate_max_eigen_value(x, A);
        y = A*x;
        x = y/norm(y);
        max_eigen_value = calculate_max_eigen_value(x, A) #uproszczony wzor bez mianownika poniewaz dlugosc wetkora x jest = 1 ze wzgledu na wprowadzoną nrmalizację w pętli

        curr_acc = abs(max_eigen_value/prev_max_val);
        if curr_acc < acc+1 && curr_acc > 1
            break
        end
    end
    
    return [max_eigen_value, x]
end


power_max_eigen (generic function with 2 methods)

In [43]:
# deflacja
max_eigen, x = power_max_eigen(A);
println(x)
println(max_eigen)

An = A - max_eigen*x*x';
sec_max_eigen, x = power_max_eigen(An)
println(x)
println(sec_max_eigen)

Ann = An - sec_max_eigen*x*x';
third_max_eigen, x = power_max_eigen(Ann)
println(x)
println(third_max_eigen)


[0.46365587190028185, 0.6382407630306943, 0.6145502102012385]
7.831586613736695
[-0.42878257244942775, -0.4453490568340997, 0.7860087296847467]
3.321327095098886
[-0.7753464274567067, 0.6279519421093321, -0.06718836083161735]
-2.1529137091165995


In [6]:
# obliczanie równan wysokiego rzedu, wyznaczanie współczynnika uwarunkowania macierzy (jak niewielka zmiana macierzy A zmienia wynik Ax=b),
# poszukiwanie promienia spektralnego - największa wartość wlasna macierzy co do modułu

function power_eigen(A::Matrix, acc=1e-10::Float64)::Vector
n = size(A, 1)
x = [1:n;];
max_eigen_value = 1.0;
min_eigen_value = 1.0;
promien_spektr = 1.0;
    
    while true
        prev_max_val = calculate_max_eigen_value(x, A);
        y = A*x;
        promien_spektr = norm(y);
        x = y/promien_spektr;
        max_eigen_value = calculate_max_eigen_value(x, A) #uproszczony wzor bez mianownika poniewaz dlugosc wetkora x jest = 1 ze wzgledu na wprowadzoną nrmalizację w pętli

        curr_acc = abs(max_eigen_value/prev_max_val);
        if curr_acc < acc+1 && curr_acc > 1
            break
        end
    end   
    
    return [max_eigen_value, min_eigen_value, promien_spektr]
end

max_val,min_val, promien = power_eigen(B);